# Snake Multiplayer

In [324]:
%serialconnect

Found serial ports: /dev/cu.usbserial-0001, /dev/cu.AVSamsungSoundbarT4-Ser, /dev/cu.Bluetooth-Incoming-Port, /dev/cu.Jakobthesuperbrain, /dev/cu.OnkyoCR-265, /dev/cu.wlan-debug 
Connecting to --port=/dev/cu.usbserial-0001 --baud=115200 
Ready.


In [325]:
import machine
import display
import random

from time import sleep

display = display.Display()

In [326]:
class Coordinate:
    x: int
    y: int

    def __init__(self, x, y):
        self.x = x
        self.y = y


In [327]:
class Playground:
    DISPLAY_WIDTH = 128
    DISPLAY_HEIGHT = 64

    BLOCK_SIZE = 4

    GAME_WIDTH = int(DISPLAY_WIDTH / BLOCK_SIZE)
    GAME_HEIGHT = int(DISPLAY_HEIGHT / BLOCK_SIZE)

    def __init__(self):
        pass

    def draw_coordinate(self, coordintate: Coordinate):
        display.fill_rect(coordintate.x * self.BLOCK_SIZE, coordintate.y * self.BLOCK_SIZE, self.BLOCK_SIZE , self.BLOCK_SIZE)

In [ ]:
class Apple:
    playground: Playground
    coordinate: Coordinate

    def __init__(self, playground):
        self.playground = playground
        x = random.randint(0, self.playground.GAME_WIDTH - 1)
        y = random.randint(0, self.playground.GAME_HEIGHT - 1)
        self.coordinate = Coordinate(x, y)
    
    def draw_apple(self):
        self.playground.draw_coordinate(self.coordinate)

In [329]:
class Snake:
    playground: Playground

    def __init__(self, playground: Playground, startPos: Coordinate):
        self.tails = []
        self.length = 10
        self.direction = 2 # 0 = up, 1 down, 2 left, 3 right

        self.tails.append(startPos)
        print(self.tails)
        self.playground = playground

    def change_direction(self, newDirection):
        self.direction = newDirection

    def moove_snake(self):
        head = self.tails[-1]
        newPos = Coordinate(head.x, head.y)

        if (self.direction == 0):
            newPos.y = head.y - 1
            if newPos.y < 0:
                newPos.y = self.playground.GAME_HEIGHT - 1
        elif (self.direction == 1):
            newPos.y = head.y + 1
            if newPos.y >= self.playground.GAME_HEIGHT:
                newPos.y = 0
        elif (self.direction == 2):
            newPos.x = head.x - 1
            if newPos.x < 0:
                newPos.x = self.playground.GAME_WIDTH - 1
        elif (self.direction == 3):
            newPos.x = head.x + 1
            if newPos.x >= self.playground.GAME_WIDTH:
                newPos.x = 0

        if (len(self.tails) >= self.length):
            self.tails.pop(0)

        self.tails.append(newPos)

    def detect_apple_colision(self, apple: Apple) -> bool:
        collisionDetected = self.detect_colision(apple.coordinate)
        if collisionDetected:
            self.length += 2
    
        return collisionDetected
    
    def detect_game_over(self):
        head = self.tails[-1]
        
        for pos in self.tails[0:-1]:
            if pos.x == head.x and pos.y == head.y:
                for i in range(0, max(5, len(self.tails) - 5)):
                    self.tails.pop()
                self.length = 5


    def detect_colision(self, coordinate) -> bool:
        head = self.tails[-1]
        return head.x == coordinate.x and head.y == coordinate.y

    def draw_snake(self):
        for tail in self.tails:
            self.playground.draw_coordinate(tail)

In [330]:
playground = Playground()

snake = Snake(playground, Coordinate(0, 0))
snake2 = Snake(playground, Coordinate(10, 20))

apple = Apple(playground)

downBtn = Pin(14, Pin.IN, Pin.PULL_UP)
rightBtn = Pin(21, Pin.IN, Pin.PULL_UP)
leftBtn = Pin(18, Pin.IN, Pin.PULL_UP)
upBtn = Pin(25, Pin.IN, Pin.PULL_UP)

while True:
    if downBtn.value() == 0:
        snake.change_direction(1)#
    if rightBtn.value() == 0:
        snake.change_direction(3)
    if leftBtn.value() == 0:
        snake.change_direction(2)
    if upBtn.value() == 0:
        snake.change_direction(0)

    display.clear()
    apple.draw_apple()
    snake.moove_snake()
    snake.draw_snake()
    if snake.detect_apple_colision(apple):
        apple = Apple(playground)
    snake.detect_game_over()

    snake2.moove_snake()
    snake2.draw_snake()
    if snake2.detect_apple_colision(apple):
        apple = Apple(playground)
    snake2.detect_game_over()

    display.show()
    sleep(0.1)

[<Coordinate object at 3fffb950>]
[<Coordinate object at 3fffbc00>]
..

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 38, in <module>
KeyboardInterrupt: 
